# Machine Learning - Assignment 2

## Decision tree induction algorithm for classification tasks

The aim of the assignment is to:

* Implement a decision tree induction algorithm for classification tasks.
* Make sure it works for real valued features and nominal features (categorical features without rank, e.g., red - blue - green).
* Test the algorithm on 3 datasets.

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries.

**IMPORTANT NOTE:** You may not import any more libraries than the ones already imported!

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Decision tree model

The main objective is to implement the decision tree model. The implemented decision tree needs to be recursive model, that is, it should be implemented general enough to call itself in order to grow. "Growing" a tree refers to the same thing as "training" a model.

As said in the introduction, the structure is set up to help with implementation, but the nature of this model makes it a bit harder to implement function-by-function. You will most likely go back and forth between these first tasks.

### 1) Grow Tree

We will start with the main function of the decision tree, the "growing" function. 

This function should be called when creating the model, but also from inside itself. It is responible for creating all the nodes and leafs in the tree.

In [ ]:
def grow_tree(data, depth=0, max_depth=10):
    # TODO: Implement the rest of this function.
    # NOTE: You will need to come back to this function to make use of later functions.

    # Check stopping criteria and create a leaf if any is true
    # if all data are of the same class it is homogeneous
    if homogeneous(data) or is_max_depth(depth, max_depth) or min_samples_split(data) or unbalanced(data):
        leaf_value = label(data)
        return {'leaf_value': leaf_value}

    best_feature_idx, best_threshold = best_split(
        data
    )  # split on feature that creates most homogenous data subsets.
    if best_feature_idx is None:
        # Could not find a better split
        leaf_value = label(data)
        return {'leaf_value': leaf_value}

    left_data, right_data = split_data(data, best_feature_idx, best_threshold)
    if len(left_data) != 0:
        left_subtree = grow_tree(left_data, depth+1, max_depth)
    else:
        leaf_value = label(data)
        return {"leaf_value": leaf_value}  # Don't grow the tree

    if len(right_data) != 0:
        right_subtree = grow_tree(right_data, depth+1, max_depth)
    else:
        leaf_value = label(data)
        return {"leaf_value": leaf_value}  # Don't grow the tree

    # Return a node
    return {
        "feature_idx": best_feature_idx,
        "threshold": best_threshold,
        "left": left_subtree,
        "right": right_subtree,
    }

In [248]:
def label(data):
    unique_classes, counts = np.unique(data[:, -1], return_counts=True)
    data[:,-1] = int(unique_classes[np.argmax(counts)])
    return data

### 2) Growth stopping conditions (or stopping criterias)

The "grow_tree" function needs some way of stop growing, otherwise it will grow indefinitely. We will adress this issue here.

The trees stopping criterias needs to handle the following:

1) When a node has only datapoints of a single class.

2) Prevent the tree from growing to large, i.e., a max depth.

3) Prevent the tree nodes from becoming to small.

4) Prevent the tree from growing when the node is large (has a lot of datapoints) but it is very unbalanced. This is an extention to case 1.

Can you think of some other stopping criterias that is good to have? 

- Minimum information gain threshold (stop if the split doesn't significantly reduce impurity).  
- Time or memory constraints (stop if training exceeds a limit).  
- Early stopping based on validation performance (prevent overfitting once performance plateaus).  
- Leaf purity threshold (stop if further splits yield minimal improvement).

In [79]:
# TODO: Change the name of the functions and implement them as you see fit.
# 1. When a node has only datapoints of a single class.
def homogeneous(data):
    class_labels = data[:, -1]
    # NOTE: it is up to me to decide if all labels has to be of the same class or for example 90% to prevent overfitting and too deep and specialized trees.

    # Check if all class labels are the same
    is_homogeneous = len(np.unique(class_labels)) == 1
    return is_homogeneous

# 2. Prevent the tree from growing to large, i.e., a max depth.
def is_max_depth(depth, max_depth):
    return depth >= max_depth

# 3. Prevent the tree nodes from becoming to small.
def min_samples_split(data):
    return len(data) < 2

# 4. Prevent the tree from growing when the node is large (has a lot of datapoints) but it is very unbalanced. This is an extension to case 1.
def unbalanced(data, size_threshold=100, ratio_threshold=0.9):
    class_labels = data[:, -1]
    label_counts = np.bincount(class_labels.astype(int))
    return len(class_labels) > size_threshold and (label_counts.max() / len(class_labels)) >= ratio_threshold

# Add more stopping criteria if needed. Don't forget to use them when growing the tree!

### 3) Best feature for splitting nodes

When we are growing the tree, we need to decide how we are going to split a node into two new nodes. This is achived by looking at the features of the data in the node and calculate the best feature to split on.

Here you have a choice:

* Split using **Information Entropy**
* Split using **Gini Impurity**

Finish the function below using Information Entropy or Gini Impurity.

**Note:** Your code should be able to handle both real and categorical features!

In [ ]:
def information_entropy(subsets):
    pass

In [377]:
def gini_impurity(labels):
    size_labels = len(labels)
    if size_labels == 0:
        return 0

    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts / size_labels
    return 1 - np.sum(probabilities**2)

### 4) Split data

When growing the tree, we need to split the data multiple times, and what we decide to split varies a lot. It is similar to splitting data into train and test sets (remember from assignment 1), but we split the data based on the best feature for growing a good tree.

**IMPORTANT NOTE:** To calculate binary splits for real-valued features, the following rule must be applied: an instance with a feature value lower than the mean feature value follows the left edge from the split node while all other instances follow the right edge from the split node.

In [379]:
def best_split(data):
    imp_min = 1
    best_feature_idx = best_threshold = None
    n_features = data.shape[1] - 1
    current_impurity = gini_impurity(data[:, -1])
    # split data into subsets with values from features
    for feature_idx in range(n_features):
        feature_col = data[:, feature_idx]
        # for feature, feature_idx in zip(features, range(n_features)):
        if isinstance(data[0][feature_idx], (int, float, np.float64)):
            threshold = np.mean(feature_col)
            left_mask = feature_col < threshold
            right_mask = ~left_mask

            left_labels = data[left_mask, -1]
            right_labels = data[right_mask, -1]

        else:
            pass
        gini_left = gini_impurity(left_labels)
        gini_right = gini_impurity(right_labels)

        weighted_left = len(left_labels) / len(data)
        weighted_right = len(right_labels) / len(data)
        weighted_gain = current_impurity - (weighted_left*gini_left + weighted_right*gini_right)
        if weighted_gain < imp_min:
            imp_min = weighted_gain
            best_feature_idx = feature_idx
            best_threshold = threshold
    return best_feature_idx, best_threshold 

In [381]:
def split_data(data, feature_idx, threshold):
    # TODO: Implement the rest of this function.
    feature_col = data[:, feature_idx]

    if isinstance(data[0][feature_idx], (int, float, np.float64)):
        left_mask = feature_col < threshold
        right_mask = ~left_mask
    else:
        # categorical
        pass
    left_data = data[left_mask]
    right_data = data[right_mask]
    return left_data, right_data

### 5) Predict with tree model

Finally, when we have grown our tree, we would like to use it for prediction. When using the tree for prediction, we traverse the tree for each datapoint untill we land in a leaf node.

In [ ]:
def predict_with_tree(data, test, tree):
    # TODO: Implement the rest of this function.
    # NOTE: This function should also be recursive.
    if 'leaf_value' in tree:
        return tree['leaf_value']

    feature_idx = tree['feature_idx']
    threshold = tree['threshold']

    if isinstance(data[0][feature_idx], (int, float, np.float64)):
        if test[feature_idx] < threshold:
            return predict_with_tree(data, test, tree['left'])
        else:
            return predict_with_tree(data, test, tree["right"])
    else:
        # categorical
        pass

## Test decision tree model, compare with scikit learn, and plot dataset results

In the last part of the lab, you are going to test your tree code and compare it to scikit learn. The goal is not to be better than an established library, but to give you an indication about if you are on the right track.

You will need to plot the results from your model and the scikit learn model using matplotlib. We suggest a simple but informative bar-charts.

To make the comparison fair, you should train and test both your decision tree algorithm and the scikit learn at least 5 times, and shuffle the data each time before splitting the data into a train and test set.

The datasets are:

* Wine - (https://archive.ics.uci.edu/dataset/109/wine)
* Heart disease - (https://archive.ics.uci.edu/dataset/45/heart+disease)
* Car - (https://archive.ics.uci.edu/dataset/19/car+evaluation)

**IMPORTANT NOTE 1:** Take note of the feature types in the datasets, some features are numerical in value but are in fact categorical features. Be sure to handle these features correctly!

**IMPORTANT NOTE 2:** In this assignment it helps to add an additional header with information about the features and if they are nominal (n) or real (r) features.

In [2]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

You may use the "**accuracy_score**" function from scikit learn (imported above) to compare the performance of your own and scikit learns models.

See below for an example use.

In [4]:
y_true = [1,1,1,1,1] # Pretend labels
y_pred = [1,1,2,2,1] # Pretend prediction

accuracy_score(y_true, y_pred)

0.6

### 6) Dataset 1: Wine

In [280]:
data_wine = pd.read_csv("wine.csv", skiprows=[1]).to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 7) Dataset 2: Heart Disease

In [6]:
data_heart = pd.read_csv("heart.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 8) Dataset 3: Car

In [7]:
data_car = pd.read_csv("car.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 9) Training with normalized data on the wine-dataset

So far, we have trained our decision trees with "raw" data, i.e., we haven't done much preprocessing on the data.

Here we will do minor preprocessing on the data with the help of the scikit-learn library: https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
from sklearn import preprocessing

# TODO: Use the wine dataset from above and scale its features and labels between 0 and 1

# TODO: Run the code from the dataset and compare the preprocessed vs non-preprocessed data.
# NOTE: You can copy most of the workflow from the dataset code above to save you some time.
# NOTE: Use the same train/test split for your tree model for both the preprocessed vs non-preprocessed data.

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is growing the tree indefinitely such a bad idea? The performance would increase would it not?

2) Beside preventing the tree from growing to large, what is the purpose of 'stopping criterias'?

3) What is the difference between **Information Entropy** and **Gini Impurity**?

4) What are some pros about using decision trees?

5) Did preprocessing the data help with performance when using decision trees?

# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!